![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [3]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

filename_features = "Dados/UCI HAR Dataset/features.txt"
filename_labels = "Dados/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "Dados/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "Dados/UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "Dados/UCI HAR Dataset/train/y_train.txt"

filename_subtest = "Dados/UCI HAR Dataset/test/subject_test.txt"
ffilename_xtest = "Dados/UCI HAR Dataset/test/X_test.txt"
filename_ytest = "Dados/UCI HAR Dataset/test/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#").squeeze("columns")
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])
("columns")

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id']).squeeze("columns")
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id']).squeeze("columns")
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [4]:
%%time
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train)

Wall time: 27.1 ms


In [11]:
%%time

clf = DecisionTreeClassifier(random_state=1234, ccp_alpha=0.001).fit(X_train, y_train)

clf=DecisionTreeClassifier(random_state=1234, ccp_alpha=0.001).fit(X_train, y_train)

print(f'Acurácia na base de teste: {clf.score(X_test, y_test)*100:.1f}\n')
print(f'Acurácia na base de treinamento: {clf.score(X_train, y_train)*100:.1f}')


Acurácia na base de teste: 85.8

Acurácia na base de treinamento: 98.1
Wall time: 5.19 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [13]:
%%time

prcomp = PCA(n_components=1).fit(X_train)

pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)

print(f'Dimensões da base de treinamento: {pc_treino.shape}')
print(f'Dimensões da base de teste: {pc_teste.shape}')

clf = DecisionTreeClassifier(random_state=1234, ccp_alpha=0.001)
clf.fit(pc_treino, y_train)

print(f'Acurácia na base de treinamento: {clf.score(pc_treino, y_train)*100:.1f}')
print(f'Acurácia na base de teste: {clf.score(pc_teste, y_test)*100:.1f}\n')

Dimensões da base de treinamento: (5514, 1)
Dimensões da base de teste: (2947, 1)
Acurácia na base de treinamento: 49.7
Acurácia na base de teste: 45.0

Wall time: 108 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [15]:
%%time

components = [1,2,5,10,50]

for n in components:
    prcomp = PCA(n_components=n).fit(X_train)

    pc_treino = prcomp.transform(X_train)
    pc_teste  = prcomp.transform(X_test)
    
    print(f'Dimensões da base de treinamento: {pc_treino.shape}')
    print(f'Dimensões da base de teste: {pc_teste.shape}')
    
    clf = DecisionTreeClassifier(random_state=1234, ccp_alpha=0.001)
    clf.fit(pc_treino, y_train)

    print(f'Acurácia na base de treinamento: {clf.score(pc_treino, y_train)*100:.1f}')
    print(f'Acurácia na base de teste: {clf.score(pc_teste, y_test)*100:.1f}\n')

Dimensões da base de treinamento: (5514, 1)
Dimensões da base de teste: (2947, 1)
Acurácia na base de treinamento: 49.7
Acurácia na base de teste: 45.0

Dimensões da base de treinamento: (5514, 2)
Dimensões da base de teste: (2947, 2)
Acurácia na base de treinamento: 62.1
Acurácia na base de teste: 57.7

Dimensões da base de treinamento: (5514, 5)
Dimensões da base de teste: (2947, 5)
Acurácia na base de treinamento: 85.1
Acurácia na base de teste: 79.0

Dimensões da base de treinamento: (5514, 10)
Dimensões da base de teste: (2947, 10)
Acurácia na base de treinamento: 90.1
Acurácia na base de teste: 80.5

Dimensões da base de treinamento: (5514, 50)
Dimensões da base de teste: (2947, 50)
Acurácia na base de treinamento: 93.2
Acurácia na base de teste: 81.5

Wall time: 932 ms


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

> Ao reduzir a dimensionalidade utilizando a técnica de Análise de Componentes Principais (PCA), observamos que a principal mudança e mais perceptível ocorre em relação ao tempo de processamento. A árvore de decisão executada com o conjunto de dados completo levou aproximadamente 5.19 segundos para ser executada, resultando em uma acurácia de 85.8% na base de teste. No entanto, ao utilizar o PCA com apenas um componente, o tempo de processamento foi reduzido para 108 ms, porém a acurácia na base de testes foi de 45%.

> Aumentando o número de componentes, foi possível aumentar gradualmente a acurácia na base de testes. Com 50 componentes, foi atingida uma acurácia de até 81.5%, com um tempo de processamento de aproximadamente 932 ms para executar todas as combinações de números de componentes. Portanto, podemos concluir que é possível obter uma acurácia semelhante à árvore com todas as variáveis do conjunto de dados, porém com um tempo de processamento reduzido em menos da metade.